In [1]:
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset("glue", 'sst2')


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding

col_to_delete = ['question1','sentence2']

def preprocessing_function(examples):
    return tokenizer(examples['sentence'], truncation=True, max_length=128)
    
tokenized_datasets = raw_datasets.map(preprocessing_function, batched=True)


# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random





In [5]:

import leader
from tqdm.auto import tqdm    
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Example text input
input_text = tokenized_datasets['train']['sentence'][0:200]
'''
input_text = []
for i in tqdm(range(len(tokenized_datasets['train']['premise'][0:200]))):
    st = tokenized_datasets['train']['premise'][i] + tokenized_datasets['train']['hypothesis'][i]
    input_text.append(st)
    '''
input_ids = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)['input_ids']


# Get the embedding of the input text
with torch.no_grad():
    embeddings = model.roberta.embeddings(input_ids)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Apply PEFT with the custom row method to replace layers
leader.PEFT_prunig(model, method='column', rank=3, input_data=embeddings, descending=False)
#targets=['key', 'value', 'dense', 'query'])
# method = 'row', 'column', 'random'

In [7]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [8]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.00,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,0.670500,0.397452,0.848336,0.840469,0.840595,0.841743
200,0.398600,0.266502,0.898116,0.892365,0.892824,0.893349
300,0.313300,0.257506,0.900864,0.894618,0.895095,0.895642
400,0.325600,0.233832,0.912963,0.913067,0.912842,0.912844
500,0.340400,0.237777,0.908839,0.900164,0.900699,0.901376
600,0.311500,0.291823,0.894566,0.883935,0.884348,0.885321
700,0.290800,0.203637,0.925564,0.925328,0.925415,0.925459
800,0.304700,0.208579,0.923351,0.922992,0.923112,0.923165
900,0.313800,0.200333,0.930239,0.929875,0.929997,0.930046
1000,0.288900,0.225876,0.926005,0.925160,0.925376,0.925459


TrainOutput(global_step=12630, training_loss=0.21138816642459196, metrics={'train_runtime': 630.9595, 'train_samples_per_second': 320.222, 'train_steps_per_second': 20.017, 'total_flos': 16257300198672.0, 'train_loss': 0.21138816642459196, 'epoch': 3.0})